# The entire Code is taken and adapted from the following sources :
1. "https://github.com/yao8839836/kg-bert"
2. "https://huggingface.co/docs/transformers/v4.25.1/en/index"
3. "https://huggingface.co/docs/datasets/index"
4. "https://huggingface.co/docs/tokenizers/index"

# **Set Up**

In [ ]:
print('Mounting google drive...')
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive"
!mkdir KG-Models
%cd "/content/drive/MyDrive/KG-Models"

Mounting google drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
mkdir: cannot create directory ‘KG-Models’: File exists
/content/drive/MyDrive/KG-Models


In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec 17 12:56:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    55W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# progress bars in model download and training scripts
!pip install tqdm~=4.64.0
# Accessing files from S3 directly.
!pip install boto3
# Used for downloading models over HTTP
!pip install requests
# bert

# wikipedia
!pip install wikipedia~=1.4.0
!pip install transformers~=4.21.1
!pip install tqdm
!pip install ujson
!pip install seqeval
!pip install sentencepiece
!pip install datasets
from datasets import Dataset, DatasetDict
from datasets import load_metric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
torch.cuda.is_available()

True

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#pip install optuna/sigopt/wandb/ray[tune] 

In [ ]:
!pip install wandb
import wandb
wandb.login()
#1c936ec91786bdd6cc3afad6d11df354cf92fbdc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anvesha. Use `wandb login --relogin` to force relogin


True

In [ ]:
%env WANDB_WATCH=all

env: WANDB_WATCH=all


In [ ]:
#The entire Code is taken and adapted from the following source :"https://github.com/yao8839836/kg-bert"
from __future__ import absolute_import, division, print_function

import argparse
import csv
import logging
import os
import random
import sys

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from torch.nn import CrossEntropyLoss, MSELoss
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
from sklearn import metrics

from transformers import AdamW, WEIGHTS_NAME, CONFIG_NAME
from transformers import DebertaTokenizer, DebertaForSequenceClassification,DebertaConfig
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import get_linear_schedule_with_warmup
#from pytorch_transformers.optimization import WarmupLinearSchedule


os.environ['CUDA_VISIBLE_DEVICES'] = '6'
#torch.backends.cudnn.deterministic = True

logger = logging.getLogger(__name__)



class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            text_c: (Optional) string. The untokenized text of the third sequence.
            Only must be specified for sequence triple tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        #Changing Here
        self.text = text
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self, data_dir):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class KGProcessor(DataProcessor):
    """Processor for knowledge graph data set."""

    def __init__(self):
        self.labels = set()

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train", data_dir)

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev", data_dir)

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "test.tsv")), "test", data_dir)

    def get_relations(self, data_dir):
        """Gets all labels (relations) in the knowledge graph."""
        # return list(self.labels)
        with open(os.path.join(data_dir, "relations.txt"), 'r') as f:
            lines = f.readlines()
            relations = []
            for line in lines:
                relations.append(line.strip())
        return relations

    def get_labels(self, data_dir):
        """Gets all labels (0, 1) for triples in the knowledge graph."""
        return ["0", "1"]

    def get_entities(self, data_dir):
        """Gets all entities in the knowledge graph."""
        # return list(self.labels)
        with open(os.path.join(data_dir, "entities.txt"), 'r') as f:
            lines = f.readlines()
            entities = []
            for line in lines:
                entities.append(line.strip())
        return entities

    def get_train_triples(self, data_dir):
        """Gets training triples."""
        return self._read_tsv(os.path.join(data_dir, "train.tsv"))

    def get_dev_triples(self, data_dir):
        """Gets validation triples."""
        return self._read_tsv(os.path.join(data_dir, "dev.tsv"))

    def get_test_triples(self, data_dir):
        """Gets test triples."""
        return self._read_tsv(os.path.join(data_dir, "test.tsv"))

    def _create_examples(self, lines, set_type, data_dir):
        """Creates examples for the training and dev sets."""
        # entity to text
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in ent_lines:
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    # creating key value pair for the ent2text dictionary
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())  # keys are treated as entities

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]

        lines_str_set = set(['\t'.join(line) for line in lines])
        examples = []
        for (i, line) in enumerate(lines):

            head_ent_text = ent2text[line[0]]
            tail_ent_text = ent2text[line[2]]
            relation_text = rel2text[line[1]]

            if set_type == "dev" or set_type == "test":
                triple_label = line[3]
                if triple_label == "1":
                    label = "1"
                else:
                    label = "0"

                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text
                #Changing Here
                text="[CLS] "+text_a+" [SEP] "+text_b+" [SEP] "+text_c + " [SEP]"
                self.labels.add(label)
                #Changing Here
                examples.append(
                    InputExample(guid=guid, text=text, label=label))

            elif set_type == "train":
                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text
                #Changing Here
                text="[CLS] "+text_a+" [SEP] "+text_b+" [SEP] "+text_c+ " [SEP]"
                #Changing Here
                examples.append(
                    InputExample(guid=guid, text=text, label="1"))

                rnd = random.random()
                guid = "%s-%s" % (set_type + "_corrupt", i)
                if rnd <= 0.5:
                    # corrupting head
                    tmp_head = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[0])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_head = random.choice(tmp_ent_list)
                        tmp_triple_str = tmp_head + \
                            '\t' + line[1] + '\t' + line[2]
                        if tmp_triple_str not in lines_str_set:
                            break
                    tmp_head_text = ent2text[tmp_head]
                    #Changing Here
                    tmp_text="[CLS] "+tmp_head_text+" [SEP] "+text_b+" [SEP] "+text_c+" [SEP]"
                    #Changing Here
                    examples.append(
                        InputExample(guid=guid, text=tmp_text, label="0"))
                else:
                    # corrupting tail
                    tmp_tail = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[2])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_tail = random.choice(tmp_ent_list)
                        tmp_triple_str = line[0] + \
                            '\t' + line[1] + '\t' + tmp_tail
                        if tmp_triple_str not in lines_str_set:
                            break
                    tmp_tail_text = ent2text[tmp_tail]
                    #Changing Here
                    tmp_text="[CLS] "+text_a+" [SEP] "+text_b+" [SEP] "+tmp_tail_text+" [SEP]"
                    #Changing Here
                    examples.append(
                        InputExample(guid=guid, text=tmp_text, label="0"))
        return examples



In [ ]:
data_dir = "/content/drive/MyDrive/kg-bert/data_sample_classifier/FB13"
processors = {
        "kg": KGProcessor,
    }
processor = processors['kg']()


In [ ]:
train_examples=processor.get_train_examples(data_dir)

In [ ]:
label_list = processor.get_labels(data_dir)
num_labels = len(label_list)

label_to_id = {v: i for i, v in enumerate(label_list)}

In [ ]:
object_dict={'guid':[id.guid for id in train_examples],'triple_texts':[x.text for x in train_examples],'labels':[label_to_id[l.label] for l in train_examples]}

In [ ]:
dataset_train = Dataset.from_dict(object_dict)
dataset_train
dataset_train[0]

{'guid': 'train-0',
 'triple_texts': "[CLS] Jean-Antoine Brutus Menier (May 17, 1795 – December 19, 1853) was a French entrepreneur and founder of the Menier family of chocolatiers. Born in Germain-de-Bourgeuil, Indre-et-Loire, he was the third child in a family of merchants. In 1811 Antoine Brutus Menier was enrolled in the La Flèche Military Academy where he studied the composition of pharmaceuticals.  The following year the 17-year-old served with the medical staff of La Grande Armée in Napoleon's invasion of Russia. In 1813 he was employed at Val-de-Grâce military hospital in Paris. In 1816, Antoine Brutus Menier married Marie-Edmée Virginie Pichon. With her sizeable dowry he was able to go into business and he established the Menier Hardware Company in the Marais district of Paris. Although not certified as a pharmacist, Menier began preparing and selling a variety of powders for medicinal purposes. This aspect of the business grew rapidly and in 1825 he began an expansion through

In [ ]:
dataset_train

Dataset({
    features: ['guid', 'triple_texts', 'labels'],
    num_rows: 50000
})

In [ ]:
eval_examples=processor.get_dev_examples(data_dir)
object_dict_eval={'guid':[id.guid for id in eval_examples],'triple_texts':[x.text for x in eval_examples],'labels':[label_to_id[l.label] for l in eval_examples]}

In [ ]:
test_examples=processor.get_test_examples(data_dir)
object_dict_test={'guid':[id.guid for id in test_examples],'triple_texts':[x.text for x in test_examples],'labels':[label_to_id[l.label] for l in test_examples]}

In [ ]:
dataset_eval = Dataset.from_dict(object_dict_eval)
dataset_eval[0]

{'guid': 'dev-0',
 'triple_texts': '[CLS] William Cooley (1783–1863) was one of the first American settlers, and a regional leader, in what is now known as Broward County in the state of Florida. His family was killed by Seminoles in 1836, during the Second Seminole War. The attack, known as the "New River Massacre", caused immediate abandonment of the area by whites.Cooley was born in Maryland, but little else is known about his life prior to 1813, when he arrived in East Florida, a province of Spanish Florida, as part of a military expedition. He established himself as a farmer in the northern part of the province before moving south, where he traded with local Indians and continued to farm. During the period in which the region was transferred from Spanish to U.S. governance, he sided with natives in a land dispute against a merchant who had received a large grant from the King of Spain and was evicting the Indians from their lands. Unhappy with the actions of the Spanish, he moved 

In [ ]:
dataset_test = Dataset.from_dict(object_dict_test)
dataset_test[0]

{'guid': 'test-0',
 'triple_texts': "[CLS] John Glenn Beall Jr. (June 19, 1927 – March 24, 2006), known as J. Glenn Beall Jr., was a Republican member of the United States Senate, representing the State of Maryland 1971–1977. He was also a member of the Maryland House of Delegates from 1962 to 1968, and the U.S. House of Representatives from the sixth district of Maryland from 1969 to 1971. His father, James Glenn Beall, was also a senator and representative from Maryland. [SEP] nationality [SEP] The United States of America (USA), commonly known as the United States (U.S. or US) or America, is a country comprising 50 states, a federal district, five major self-governing territories, and various possessions. At 3.8 million square miles (9.8 million km2), the United States is the world's third or fourth largest country by total area and is slightly smaller than the entire continent of Europe's 3.9 million square miles (10.1 million km2). With a population of more than 327 million people

In [ ]:
from datasets import DatasetDict
dataset = DatasetDict({"train":dataset_train,"eval":dataset_eval,"test":dataset_test})
dataset

DatasetDict({
    train: Dataset({
        features: ['guid', 'triple_texts', 'labels'],
        num_rows: 50000
    })
    eval: Dataset({
        features: ['guid', 'triple_texts', 'labels'],
        num_rows: 4186
    })
    test: Dataset({
        features: ['guid', 'triple_texts', 'labels'],
        num_rows: 16770
    })
})

# **Deberta Implementation**

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!stat cuda

  File: cuda -> /etc/alternatives/cuda
  Size: 22        	Blocks: 0          IO Block: 4096   symbolic link
Device: 3dh/61d	Inode: 2244232     Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-12-13 14:28:53.000000000 +0000
Modify: 2022-12-13 14:28:53.000000000 +0000
Change: 2022-12-13 15:32:19.189330995 +0000
 Birth: -


In [ ]:
# Use PyTorch to check versions, CUDA version and cuDNN

import torch

print("PyTorch version: ")
print(torch.__version__)
print("CUDA Version: ")
print(torch.version.cuda)
print("cuDNN version is: ")
print(torch.backends.cudnn.version())

PyTorch version: 
1.12.1+cu116
CUDA Version: 
11.6
cuDNN version is: 
8302


In [ ]:
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base", truncation=True, padding='max_length', max_length=115,return_tensors='pt')

In [ ]:
encoded_dataset = dataset.map(lambda x: tokenizer(x['triple_texts'], truncation=True, padding='max_length', max_length=115,return_tensors='pt'), remove_columns=['guid', 'triple_texts'],batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

In [ ]:
encoded_dataset


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4186
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16770
    })
})

In [ ]:
encoded_dataset["train"].features['input_ids']

Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)

In [ ]:
from transformers import DebertaForSequenceClassification

model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=num_labels).to(device)

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.weight', 'classifi

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir='./output_FB13_deberta_tripleclassifier_trial1/',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=512,   #batch size for evaluation
    gradient_accumulation_steps=1, #   # 
    warmup_steps=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs', 
    report_to = 'wandb',                     # enable logging to W&B
    overwrite_output_dir = True,
    learning_rate = 5e-5,  
    save_strategy="no",
     # we can customize learning rate
    evaluation_strategy = 'epoch',  
    eval_steps=50,
    logging_strategy = 'epoch',            
    load_best_model_at_end = False,
    metric_for_best_model = 'accuracy',
    greater_is_better = True,
    auto_find_batch_size=True,
    run_name = 'deberta_triple_classification_final',  
            # name of the W&B run
)

In [ ]:
import numpy as np
import evaluate


#Evaluation Metric 


def compute_metrics(eval_pred):
  metric1 = evaluate.load("precision")
  metric2 = evaluate.load("recall")
  metric3 = evaluate.load("f1")
  metric4 = evaluate.load("accuracy")

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  precision = metric1.compute(predictions=predictions, references=labels,average="weighted")["precision"]
  recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
  f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
  accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

  return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}






In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, batch,return_outputs=False):
        num_labels = 2
        batch = batch.to(device) 
        # forward pass
        outputs = model(**batch)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, num_labels), batch['labels'].view(-1))
        return (loss, outputs) if return_outputs else loss



                

In [ ]:
#train_dataset_t1 = encoded_dataset["train"].shard(index=1, num_shards=10)


In [ ]:
from transformers.data.data_collator import default_data_collator
trainer = CustomTrainer(model = model,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

In [ ]:

trainer.train()
wandb.finish()

/usr/local/lib/python3.8/dist-packages/accelerate/memory_utils.py:23: FutureWarning: memory_utils has been reorganized to utils.memory. Import `find_executable_batchsize` from the main `__init__`: `from accelerate import find_executable_batch_size` to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50000
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2346
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.577800,0.586718,0.663020,0.654085,0.649279,0.654085
2,0.475400,0.529482,0.776892,0.677496,0.645701,0.677496
3,0.435100,0.502747,0.768169,0.696369,0.674587,0.696369


***** Running Evaluation *****
  Num examples = 4186
  Batch size = 512


***** Running Evaluation *****
  Num examples = 4186
  Batch size = 512
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 512


Training completed. Do not forget to share your model on huggingface.co/models =)




eval/accuracy,▁▅█
eval/f1,▂▁█
eval/loss,█▃▁
eval/precision,▁█▇
eval/recall,▁▅█
eval/runtime,█▁▃
eval/samples_per_second,▁█▆
eval/steps_per_second,▁██
train/epoch,▁▁▅▅███
train/global_step,▁▁▅▅███
train/learning_rate,█▅▁


In [ ]:
trainer.save_model(f'./output_FB15K_deberta_tripleclassification_trial1')

Saving model checkpoint to ./output_FB15K_deberta_tripleclassification_trial1
Configuration saved in ./output_FB15K_deberta_tripleclassification_trial1/config.json
Model weights saved in ./output_FB15K_deberta_tripleclassification_trial1/pytorch_model.bin
tokenizer config file saved in ./output_FB15K_deberta_tripleclassification_trial1/tokenizer_config.json
Special tokens file saved in ./output_FB15K_deberta_tripleclassification_trial1/special_tokens_map.json


In [ ]:
wandb.init()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Hyperparameter optimization

In [ ]:
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "objective", "goal": "minimize"},
        "parameters": {
            "learning_rate": {"values": [ 5e-5, 5e-3, 2e-5]},
            "per_device_train_batch_size": {"values": [64, 128]},
            "num_train_epochs": {"values": [3,6]},
        },
    }

In [ ]:

def model_init(trial):
    return DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=num_labels)

In [ ]:
from transformers.data.data_collator import default_data_collator
trainer = CustomTrainer(model_init = model_init,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

loading configuration file https://huggingface.co/microsoft/deberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e313266bff73867debdfa78c78a9a4966d5e78281ac4ed7048c178b16a37eba7.fb501413b9cef9cef6babdc543bb4153cbec58d52bce077647efba3e3f14ccf3
Model config DebertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "c2p",
    "p2c"
  ],
  "position_biased_input": false,
  "relative_attention": true,
  "transformers_version": "4.21.3",
  "type_vocab_size": 0,
  "vocab_size": 50265
}

loading weights file https://huggingface.co

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="wandb",
    hp_space=wandb_hp_space,
    n_trials=5,
    
)

wandb sweep id - 7tqe3x5v


Create sweep with ID: 7tqe3x5v
Sweep URL: https://wandb.ai/anvesha/uncategorized/sweeps/7tqe3x5v


wandb: Agent Starting Run: 9gsri7ev with config:
wandb: 	learning_rate: 5e-05
wandb: 	num_train_epochs: 3
wandb: 	per_device_train_batch_size: 128
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
W&B Sweep parameters: {'learning_rate': 5e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 128, '_wandb': {}, 'assignments': {}, 'metric': 'eval/loss'}
loading configuration file https://huggingface.co/microsoft/deberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e313266bff73867debdfa78c78a9a4966d5e78281ac4ed7048c178b16a37eba7.fb501413b9cef9cef6babdc543bb4153cbec58d52bce077647efba3e3f14ccf3
Model config DebertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.636700,0.729657,0.519547,0.519350,0.518137,0.519350
2,0.539000,0.607022,0.721954,0.618490,0.568165,0.618490
3,0.491300,0.580928,0.742537,0.642379,0.601208,0.642379


***** Running Evaluation *****
  Num examples = 4186
  Batch size = 512
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 512
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 512


Training completed. Do not forget to share your model on huggingface.co/models =)




eval/accuracy,▁▇█
eval/f1,▁▅█
eval/loss,█▂▁
eval/precision,▁▇█
eval/recall,▁▇█
eval/runtime,▄▁█
eval/samples_per_second,▅█▁
eval/steps_per_second,▆█▁
train/epoch,▁▁▅▅███
train/global_step,▁▁▅▅███
train/learning_rate,█▅▁


In [ ]:
best_trial

BestRun(run_id='xrbw7nq5', objective=2.8141996939515317, hyperparameters={'learning_rate': 2e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 64, 'assignments': {}, 'metric': 'eval/loss'})

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir='./output_FB13_deberta_tripleclassifier_trial2/',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=512,   #batch size for evaluation
    gradient_accumulation_steps=1, #   # 
    warmup_steps=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs', 
    report_to = 'wandb',                     # enable logging to W&B
    overwrite_output_dir = True,
    learning_rate = 2e-5,  
    save_strategy="no",
     # we can customize learning rate
    evaluation_strategy = 'epoch',  
    eval_steps=50,
    logging_strategy = 'epoch',            
    load_best_model_at_end = False,
    metric_for_best_model = 'accuracy',
    greater_is_better = True,
    auto_find_batch_size=True,
    run_name = 'deberta_triple_classification_final_2',  
            # name of the W&B run
)

PyTorch: setting up devices


In [ ]:
from transformers.data.data_collator import default_data_collator
'''train_dataloader=DataLoader(encoded_dataset["train"], collate_fn=default_data_collator, batch_size=args.per_device_train_batch_size)
eval_dataloader=DataLoader(encoded_dataset["eval"], collate_fn=default_data_collator, batch_size=args.per_device_eval_batch_size)
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
        model, args.optim, train_dataloader, eval_dataloader, args.lr_scheduler_type
    )'''
trainer = CustomTrainer(model = model,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

In [ ]:

trainer.train()


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50000
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2346
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.548100,0.571749,0.735127,0.634974,0.591470,0.634974
2,0.468900,0.526309,0.762397,0.681796,0.655328,0.681796
3,0.441600,0.518454,0.755185,0.693024,0.673118,0.693024


***** Running Evaluation *****
  Num examples = 4186
  Batch size = 512
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 512
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 512


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2346, training_loss=0.4862109588336863, metrics={'train_runtime': 927.4147, 'train_samples_per_second': 161.74, 'train_steps_per_second': 2.53, 'total_flos': 1.0329704271e+16, 'train_loss': 0.4862109588336863, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4186
  Batch size = 512


{'eval_loss': 0.5184540152549744,
 'eval_precision': 0.7551853595528677,
 'eval_recall': 0.6930243669374104,
 'eval_f1': 0.6731179416106718,
 'eval_accuracy': 0.6930243669374104,
 'eval_runtime': 9.929,
 'eval_samples_per_second': 421.594,
 'eval_steps_per_second': 0.906,
 'epoch': 3.0}

In [ ]:
trainer.predict(encoded_dataset['test'])

***** Running Prediction *****
  Num examples = 16770
  Batch size = 512


PredictionOutput(predictions=array([[-1.3422291 ,  0.8284955 ],
       [ 2.300912  , -2.6236713 ],
       [-0.79005915,  0.12222886],
       ...,
       [ 2.1421192 , -2.4067314 ],
       [-2.2077248 ,  2.3069701 ],
       [ 3.1935072 , -3.1596968 ]], dtype=float32), label_ids=array([1, 0, 1, ..., 0, 1, 0]), metrics={'test_loss': 0.5208362340927124, 'test_precision': 0.7525066818763942, 'test_recall': 0.6872391174716757, 'test_f1': 0.6656324034635314, 'test_accuracy': 0.6872391174716757, 'test_runtime': 32.7512, 'test_samples_per_second': 512.042, 'test_steps_per_second': 1.008})

In [ ]:
trainer.save_model(f'./output_FB13_deberta_tripleclassification_trial_3')

Saving model checkpoint to ./output_FB15K_deberta_tripleclassification_trial2
Configuration saved in ./output_FB15K_deberta_tripleclassification_trial2/config.json
Model weights saved in ./output_FB15K_deberta_tripleclassification_trial2/pytorch_model.bin
tokenizer config file saved in ./output_FB15K_deberta_tripleclassification_trial2/tokenizer_config.json
Special tokens file saved in ./output_FB15K_deberta_tripleclassification_trial2/special_tokens_map.json


# **XLNet Implementation**

In [ ]:
from transformers import XLNetTokenizer

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased", truncation=True, padding='max_length', max_length=100,return_tensors='pt')

In [ ]:
encoded_dataset = dataset.map(lambda x: tokenizer(x['triple_texts'], truncation=True, padding='max_length', max_length=130,return_tensors='pt'), remove_columns=['guid', 'triple_texts'],batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

In [ ]:
encoded_dataset


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4186
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16770
    })
})

In [ ]:
encoded_dataset["train"].features['input_ids']

Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)

In [ ]:
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels).to(device)

loading configuration file https://huggingface.co/xlnet-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346
Model config XLNetConfig {
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length"

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir='./output_FB13_xlnet_tripleclassifier/',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    warmup_steps=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs', 
    report_to = 'wandb',                     # enable logging to W&B
    overwrite_output_dir = True,
    learning_rate = 5e-5,  
    save_strategy="no",
     # we can customize learning rate
    evaluation_strategy = 'epoch',  
    eval_steps=50,
    logging_strategy = 'epoch',            
    load_best_model_at_end = False,
    metric_for_best_model = 'accuracy',
    greater_is_better = True,
    run_name = 'tripleclassifier_xlnet_trial1'            # name of the W&B run
)

In [ ]:
import numpy as np
import evaluate


#Evaluation Metric 


def compute_metrics(eval_pred):
  metric1 = evaluate.load("precision")
  metric2 = evaluate.load("recall")
  metric3 = evaluate.load("f1")
  metric4 = evaluate.load("accuracy")

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  precision = metric1.compute(predictions=predictions, references=labels,average="weighted")["precision"]
  recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
  f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
  accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

  return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}






In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, batch,return_outputs=False):
        num_labels = 2
        #batch = batch.to(device)   #Moving to gpu
        # forward pass
        outputs = model(**batch)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, num_labels), batch['labels'].view(-1))
        return (loss, outputs) if return_outputs else loss



                

In [ ]:
from transformers.data.data_collator import default_data_collator
trainer = CustomTrainer(model = model,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

In [ ]:
trainer.train()
#trainer.save_model(f'./output_FB13_xlnet_tripleclassifier/')

***** Running training *****
  Num examples = 50000
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2346
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.587400,0.596662,0.732317,0.603918,0.540416,0.603918
2,0.513300,0.575268,0.743046,0.642379,0.601072,0.642379
3,0.477200,0.573262,0.720295,0.647396,0.615595,0.647396


***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2346, training_loss=0.5260007554206295, metrics={'train_runtime': 923.6284, 'train_samples_per_second': 162.403, 'train_steps_per_second': 2.54, 'total_flos': 8346101940000000.0, 'train_loss': 0.5260007554206295, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128


{'eval_loss': 0.5732622742652893,
 'eval_precision': 0.7202945173254618,
 'eval_recall': 0.6473960821786908,
 'eval_f1': 0.6155948679087513,
 'eval_accuracy': 0.6473960821786908,
 'eval_runtime': 10.7863,
 'eval_samples_per_second': 388.083,
 'eval_steps_per_second': 3.059,
 'epoch': 3.0}

In [ ]:
trainer.predict(encoded_dataset['test'])

***** Running Prediction *****
  Num examples = 16770
  Batch size = 128


PredictionOutput(predictions=array([[-0.2289222 , -0.08057125],
       [-0.2289222 , -0.08057125],
       [-0.6417513 ,  0.30719745],
       ...,
       [ 0.94565487, -1.192969  ],
       [-2.50675   ,  2.0796537 ],
       [ 2.5390873 , -2.7706814 ]], dtype=float32), label_ids=array([1, 0, 1, ..., 0, 1, 0]), metrics={'test_loss': 0.567399263381958, 'test_precision': 0.7217610023780492, 'test_recall': 0.6483005366726297, 'test_f1': 0.6165447243027193, 'test_accuracy': 0.6483005366726297, 'test_runtime': 37.6394, 'test_samples_per_second': 445.544, 'test_steps_per_second': 3.507})

# Hyperparameter Optimisation

In [ ]:
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "objective", "goal": "minimize"},
        "parameters": {
            "learning_rate": {"values": [ 5e-5, 5e-3, 2e-5]},
            "per_device_train_batch_size": {"values": [64, 128]},
            "num_train_epochs": {"values": [3,6]},
        },
    }

In [ ]:

def model_init(trial):
    return XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels)

In [ ]:
from transformers.data.data_collator import default_data_collator
trainer = CustomTrainer(model_init = model_init,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

loading configuration file https://huggingface.co/xlnet-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346
Model config XLNetConfig {
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length"

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="wandb",
    hp_space=wandb_hp_space,
    n_trials=5,
    
)

wandb sweep id - qe2cfdxz


Create sweep with ID: qe2cfdxz
Sweep URL: https://wandb.ai/anvesha/uncategorized/sweeps/qe2cfdxz


wandb: Agent Starting Run: u38gdgmv with config:
wandb: 	learning_rate: 5e-05
wandb: 	num_train_epochs: 6
wandb: 	per_device_train_batch_size: 64
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
W&B Sweep parameters: {'learning_rate': 5e-05, 'num_train_epochs': 6, 'per_device_train_batch_size': 64, '_wandb': {}, 'assignments': {}, 'metric': 'eval/loss'}
loading configuration file https://huggingface.co/xlnet-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346
Model config XLNetConfig {
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.832700,0.735306,0.250000,0.500000,0.333333,0.500000
2,0.754600,0.725765,0.250000,0.500000,0.333333,0.500000
3,0.718600,0.693979,0.250000,0.500000,0.333333,0.500000


***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. U

eval/accuracy,▁▁▁
eval/f1,▁▁▁
eval/loss,█▆▁
eval/precision,▁▁▁
eval/recall,▁▁▁
eval/runtime,█▁▃
eval/samples_per_second,▁█▆
eval/steps_per_second,▁█▆
train/epoch,▁▁▅▅███
train/global_step,▁▁▅▅███
train/learning_rate,█▅▁


In [ ]:
best_trial

BestRun(run_id='zkpvx4cs', objective=2.6259788260681534, hyperparameters={'learning_rate': 5e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 64, 'assignments': {}, 'metric': 'eval/loss'})

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir='./output_FB13_xlnet_tripleclassifier/',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    warmup_steps=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs', 
    report_to = 'wandb',                     # enable logging to W&B
    overwrite_output_dir = True,
    learning_rate = 5e-5,  
    save_strategy="no",
     # we can customize learning rate
    evaluation_strategy = 'epoch',  
    eval_steps=50,
    logging_strategy = 'epoch',            
    load_best_model_at_end = False,
    metric_for_best_model = 'accuracy',
    greater_is_better = True,
    run_name = 'tripleclassifier_xlnet_trial2'            # name of the W&B run
)

PyTorch: setting up devices


In [ ]:
from transformers.data.data_collator import default_data_collator
'''train_dataloader=DataLoader(encoded_dataset["train"], collate_fn=default_data_collator, batch_size=args.per_device_train_batch_size)
eval_dataloader=DataLoader(encoded_dataset["eval"], collate_fn=default_data_collator, batch_size=args.per_device_eval_batch_size)
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
        model, args.optim, train_dataloader, eval_dataloader, args.lr_scheduler_type
    )'''
trainer = CustomTrainer(model = model,                  # model to be trained
    args = args,                    # training args
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["eval"],
    tokenizer=tokenizer,            # for padding batched data
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50000
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2346
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.529900,0.522175,0.721723,0.693024,0.682759,0.693024
2,0.451900,0.559307,0.764239,0.696847,0.676201,0.696847
3,0.413500,0.530595,0.760341,0.701386,0.683465,0.701386


***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128
***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2346, training_loss=0.4650932370549273, metrics={'train_runtime': 1254.8602, 'train_samples_per_second': 119.535, 'train_steps_per_second': 1.87, 'total_flos': 1.0849932522e+16, 'train_loss': 0.4650932370549273, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4186
  Batch size = 128


{'eval_loss': 0.5305948853492737,
 'eval_precision': 0.7603408218004132,
 'eval_recall': 0.7013855709507884,
 'eval_f1': 0.6834654319449166,
 'eval_accuracy': 0.7013855709507884,
 'eval_runtime': 13.3208,
 'eval_samples_per_second': 314.246,
 'eval_steps_per_second': 2.477,
 'epoch': 3.0}

In [ ]:
trainer.predict(encoded_dataset['test'])

***** Running Prediction *****
  Num examples = 16770
  Batch size = 128


PredictionOutput(predictions=array([[-0.77169037,  0.9086425 ],
       [ 3.5360355 , -3.1203113 ],
       [-0.41044062,  0.48162535],
       ...,
       [ 1.3868324 , -1.7380533 ],
       [-2.3927355 ,  2.668788  ],
       [ 3.8405056 , -3.6908524 ]], dtype=float32), label_ids=array([1, 0, 1, ..., 0, 1, 0]), metrics={'test_loss': 0.5302715301513672, 'test_precision': 0.7544968110216128, 'test_recall': 0.6968992248062016, 'test_f1': 0.6787213186947563, 'test_accuracy': 0.6968992248062016, 'test_runtime': 50.3285, 'test_samples_per_second': 333.211, 'test_steps_per_second': 2.623})

In [ ]:
trainer.save_model(f'./output_FB13_xlnet_tripleclassifier/')

Saving model checkpoint to ./output_FB13_xlnet_tripleclassifier/
Configuration saved in ./output_FB13_xlnet_tripleclassifier/config.json
Model weights saved in ./output_FB13_xlnet_tripleclassifier/pytorch_model.bin
tokenizer config file saved in ./output_FB13_xlnet_tripleclassifier/tokenizer_config.json
Special tokens file saved in ./output_FB13_xlnet_tripleclassifier/special_tokens_map.json


In [ ]:
wandb.finish()

eval/accuracy,▁▄██
eval/f1,▇▁██
eval/loss,▁█▃▃
eval/precision,▁█▇▇
eval/recall,▁▄██
eval/runtime,▇▁█▄
eval/samples_per_second,▂█▁▅
eval/steps_per_second,▂█▁▅
train/epoch,▁▁▅▅████
train/global_step,▁▁▅▅█████
train/learning_rate,█▅▁
